<a href="https://colab.research.google.com/github/ikm4rkov/OCR_examples/blob/main/passport_recognition_alpha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Определение документа на изображении, удаление 3D-деформаций и бинаризация изображения осуществляются с помощью document-scanner https://github.com/endalk200/document-scanner.
Установка через pypi не удалась, поэтому был использован source. Так как он на python, не потребовалась компиляция, только зависимости.

In [1]:
!git clone https://github.com/endalk200/document-scanner.git

Cloning into 'document-scanner'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 81 (delta 24), reused 62 (delta 17), pack-reused 0 (from 0)
Receiving objects: 100% (81/81), 20.61 MiB | 35.06 MiB/s, done.
Resolving deltas: 100% (24/24), done.


С версиями в requirements.txt зависимости не установились, поэтому жесткие версии были удалены.

In [2]:
!cat document-scanner/requirements.txt

matplotlib
numpy
ocrd-fork-pylsd
opencv-python
scipy

In [3]:
!pip install -r document-scanner/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.6 MB/s eta 0:00:00


Document-scanner можно запустить или на одном изображении (--image), или на множестве (--images). Вне зависисмости от опции, выходные файлы называются так же как входные и сохраняются в директории output, создаваемой в директории запуска.

In [4]:
!mkdir prescan

In [5]:
!python document-scanner/scan.py --images prescan/

Proccessed 8.png
Proccessed 2.png
Proccessed 5.png
Proccessed 3.png
Proccessed 4.png
Proccessed 6.png
Proccessed 1.png
Proccessed 7.png
Proccessed 9.jpg
Proccessed 10.jpg


Изображения в разном порядке, имеются дубли страниц и неодинаковые ориентации.

Ориентирование документа параллельно сторонам изображения и попытка учест по отличить повороты, кратные 90 градусам с помощью opencv. В связи с этим установка opencv-python.

In [6]:
!pip install opencv-python

In [7]:
!cat orientation.py

import cv2
import sys
import os

def orient_document(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Ошибка: изображение по пути '{image_path}' не найдено.")
        return

    _, thresh = cv2.threshold(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), 150, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    largest_contour = max(contours, key=cv2.contourArea)
    rect = cv2.minAreaRect(largest_contour)
    angle = rect[2]

    if angle < -45:
        angle += 90
    elif angle > 45:
        angle -= 90

    if abs(angle) < 5:
        print("Документ уже ориентирован.")
        rotated_image = image
    else:
        if angle > 0:
            angle = 90 - angle
        (h, w) = image.shape[:2]
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        rotated_image = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICAT

In [8]:
!for file in output/*; do python orientation.py $file; done

Изображение сохранено по пути: output/10_rotated.png
Документ уже ориентирован.
Изображение сохранено по пути: output/1_rotated.png
Документ уже ориентирован.
Изображение сохранено по пути: output/2_rotated.png
Документ уже ориентирован.
Изображение сохранено по пути: output/3_rotated.png
Документ уже ориентирован.
Изображение сохранено по пути: output/4_rotated.png
Документ уже ориентирован.
Изображение сохранено по пути: output/5_rotated.png
Документ уже ориентирован.
Изображение сохранено по пути: output/6_rotated.png
Документ уже ориентирован.
Изображение сохранено по пути: output/7_rotated.png
Документ уже ориентирован.
Изображение сохранено по пути: output/8_rotated.png
Документ уже ориентирован.
Изображение сохранено по пути: output/9_rotated.png


Распознавание структуры документа с помощью deepdoctection https://github.com/deepdoctection/deepdoctection. Работает совместно с моделью распознавания письменной речи (OCR), и предлагаются на выбор 4 модели. Из них только Tesseract и DocTR поддерживают русский, Tesseract будет использован далее как инструмент по-умолчанию.

In [9]:
!pip install deepdoctection[tf]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 45.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of onnx to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of onnx to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Установка Tesseract.

In [1]:
!sudo apt install tesseract-ocr
!sudo apt install libtesseract-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (4,649 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

Deepdoctection, похоже, работает либо с .PDF, либо с один файлом в директории, поэтому под каждый файл создается директория. Также для внутренней работы с HTML устанавливается poppler.

In [2]:
!mkdir -p output/rotated && for file in output/*_rotated*; do [[ -f "$file" ]] && base_name=$(basename "$file" | sed 's/\.[^.]*$//') && mkdir -p "output/rotated/$base_name" && cp "$file" "output/rotated/$base_name/"; done

In [3]:
!pip install poppler-utils

Для поддержки русского языка Tesseract требуется загрузить файл с данными.

In [4]:
!wget https://github.com/tesseract-ocr/tessdata/raw/refs/heads/main/rus.traineddata -P /usr/share/tesseract-ocr/4.00/tessdata/

--2024-11-04 18:56:29--  https://github.com/tesseract-ocr/tessdata/raw/refs/heads/main/rus.traineddata
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/tesseract-ocr/tessdata/refs/heads/main/rus.traineddata [following]
--2024-11-04 18:56:29--  https://raw.githubusercontent.com/tesseract-ocr/tessdata/refs/heads/main/rus.traineddata
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19920885 (19M) [application/octet-stream]
Saving to: ‘/usr/share/tesseract-ocr/4.00/tessdata/rus.traineddata’

rus.traineddata     100%[===================>]  19.00M   101MB/s    in 0.2s    

2024-11-04 18:56:29 (101 MB/s)

Сохраним результаты отпределения разметки и текста deepdoctection в одну директорию.

In [5]:
!mkdir output/deepdoctection_results

Сохраняются табличные структурные элементы и блоки текста. Далее медленный этап.

In [6]:
import deepdoctection as dd
import os
from pathlib import Path


analyzer = dd.get_dd_analyzer(config_overwrite=["LANGUAGE='rus'"])

path = Path("output/rotated")
subdirs = [d for d in path.iterdir() if d.is_dir()]

for i in subdirs:
  df = analyzer.analyze(path=i)
  df.reset_state()

  doc = iter(df)
  page = next(doc)

  base_name = os.path.splitext(page.file_name)[0]
  print(base_name)
  # Сохранение таблицы в файл
  if page.tables:
      tables_file_path = f"output/deepdoctection_results/{base_name}_tables.html"
      with open(tables_file_path, 'w') as tables_file:
          tables_file.write(page.tables[0].html)
      print(f"Таблицы сохранены в '{tables_file_path}'")
  else:
      print("Таблицы не найдены.")

  # Сохранение текста в файл
  try:
      text_content = '\n'.join(page.text_)
  except ValueError:
      text_content = ''

  text_file_path = f"output/deepdoctection_results/{base_name}_text.txt"
  with open(text_file_path, 'w') as text_file:
      text_file.write(text_content)
  print(f"Текст сохранен в '{text_file_path}'")

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
[1104 18:56.51 @utils.py:161]  INF  NumExpr defaulting to 2 threads.
[1104 18:56.51 @env_info.py:449]  WRN  Both DD_USE_TORCH and DD_USE_TF are set. Defaulting to PyTorch. If you want a different behaviour, set DD_USE_TORCH to None before importing deepdoctection.
[1104 18:56.56 @file_utils.py:36]  INF  PyTorch version 2.5.0+cu121 available.
[1104 18:56.56 @file_utils.py:74]  INF  Disabling Tensorflow because USE_TORCH is set
[1104 18:57.01 @dd.py:472]  INF  Config: 
 {'DEVICE': device(type='cpu'),
 'LANGUAGE': 'rus',
 'LAYOUT_NMS_PAIRS': {'COMBINATIONS': None, 'PRIORITY': None, 'THRESHOLDS': None},
 'LIB': 'PT',
 'OCR': {'CONFIG': {'TESSERACT': 'dd/conf_tesseract.yaml'},
         'USE_DOCTR': False,
         'USE_TESSERACT': True,
         'USE_TEXTRACT': False,
         'WEIGHTS': {'DOCTR_RECOGNITION': {'PT': 'doctr/crnn_vgg16_bn/pt/crnn_vgg16_bn-9762b0b0.pt',
                                           'TF': 'doctr/crnn_vgg1

d2_model_0829999_layout_inf_only.ts:   0%|          | 0.00/275M [00:00<?, ?B/s]

[1104 18:57.03 @model.py:1197]  ERR  File downloaded from deepdoctection/d2_casc_rcnn_X_32xd4_50_FPN_GN_2FC_publaynet_inference_only does not match the expected size! You may have downloaded a broken file, or the upstream may have modified the file.


CASCADE_RCNN_R_50_FPN_GN_TS.yaml:   0%|          | 0.00/143 [00:00<?, ?B/s]

d2_model_1639999_item_inf_only.ts:   0%|          | 0.00/275M [00:00<?, ?B/s]

...

8_rotated
Таблицы не найдены.
Текст сохранен в 'output/deepdoctection_results/8_rotated_text.txt'


|          |1/?[00:00<00:00,289.76it/s]
[1104 19:01.52 @doctectionpipe.py:84]  INF  Processing 2_rotated.png
[1104 19:02.03 @context.py:133]  INF  ImageLayoutService total: 11.0515 sec.
[1104 19:02.16 @context.py:133]  INF  SubImageLayoutService total: 13.0996 sec.
[1104 19:02.27 @context.py:133]  INF  SubImageLayoutService total: 11.2061 sec.
[1104 19:02.27 @context.py:133]  INF  TableSegmentationService total: 0.2287 sec.
[1104 19:02.27 @context.py:133]  INF  TableSegmentationRefinementService total: 0.1227 sec.
[1104 19:02.31 @context.py:133]  INF  TextExtractionService total: 3.3705 sec.
[1104 19:02.31 @context.py:133]  INF  MatchingService total: 0.0067 sec.
[1104 19:02.31 @context.py:133]  INF  TextOrderService total: 0.0796 sec.
[1104 19:02.35 @view.py:326]  WRN  html construction not possible
[1104 19:02.35 @view.py:326]  WRN  html construction not possible
[1104 19:02.35 @view.py:326]  WRN  html construction not possible
[1104 19:02.35 @view.py:326]  WRN  html construction not

2_rotated
Таблицы сохранены в 'output/deepdoctection_results/2_rotated_tables.html'
Текст сохранен в 'output/deepdoctection_results/2_rotated_text.txt'


|          |1/?[00:00<00:00,2734.23it/s]
[1104 19:02.35 @doctectionpipe.py:84]  INF  Processing 3_rotated.png
[1104 19:02.46 @context.py:133]  INF  ImageLayoutService total: 10.4101 sec.
[1104 19:02.46 @context.py:133]  INF  SubImageLayoutService total: 0.0 sec.
[1104 19:02.46 @context.py:133]  INF  SubImageLayoutService total: 0.0001 sec.
[1104 19:02.46 @context.py:133]  INF  TableSegmentationService total: 0.0001 sec.
[1104 19:02.46 @context.py:133]  INF  TableSegmentationRefinementService total: 0.0001 sec.
[1104 19:02.48 @context.py:133]  INF  TextExtractionService total: 2.4399 sec.
[1104 19:02.48 @context.py:133]  INF  MatchingService total: 0.0012 sec.
[1104 19:02.48 @context.py:133]  INF  TextOrderService total: 0.0062 sec.


3_rotated
Таблицы не найдены.
Текст сохранен в 'output/deepdoctection_results/3_rotated_text.txt'


Deepdoctection испытывает сложности с распознаванием структур без рамок (печатей, таблиц...) и без большого расстояния между блоками текста. Поэтому дополнительно к нему прилагается вывод Easy-OCR для добычи всего текста.

In [7]:
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 18.9 MB/s eta 0:00:00


In [8]:
!mkdir output/easy_ocr_output

In [9]:
import cv2
import easyocr
import os

reader = easyocr.Reader(['ru'])

input_directory = 'output'
output_directory = 'output/easy_ocr_output'
os.makedirs(output_directory, exist_ok=True)

for filename in os.listdir(input_directory):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(input_directory, filename)
        image = cv2.imread(image_path)

        if image is None:
            print(f"Ошибка: изображение по пути '{image_path}' не найдено.")
            continue


        results = reader.readtext(image)


        recognised_text = [text for (_, text, _) in results]


        base_name = os.path.splitext(filename)[0]
        output_file = os.path.join(output_directory, f"{base_name}_recognised.txt")
        with open(output_file, 'w') as f:
            for line in recognised_text:
                f.write(line + '\n')

        print(f"Распознанный текст сохранен в '{output_file}'.")

[1104 19:04.31 @easyocr.py:80]  WRN  Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
[1104 19:04.31 @easyocr.py:251]  WRN  Downloading detection model, please wait. This may take several minutes depending upon your network connection.


Progress: |██████████████████████████████████████████████████| 100.0% Complete

[1104 19:04.37 @easyocr.py:255]  INF  Download complete
[1104 19:04.37 @easyocr.py:176]  WRN  Downloading recognition model, please wait. This may take several minutes depending upon your network connection.


Progress: |██████████████████████████████████████████████████| 100.1% Complete

[1104 19:04.39 @easyocr.py:180]  INF  Download complete.


Распознанный текст сохранен в 'output/easy_ocr_output/2_rotated_recognised.txt'.
Распознанный текст сохранен в 'output/easy_ocr_output/8_recognised.txt'.
Распознанный текст сохранен в 'output/easy_ocr_output/2_recognised.txt'.
Распознанный текст сохранен в 'output/easy_ocr_output/5_recognised.txt'.
Распознанный текст сохранен в 'output/easy_ocr_output/3_recognised.txt'.
Распознанный текст сохранен в 'output/easy_ocr_output/4_recognised.txt'.
Распознанный текст сохранен в 'output/easy_ocr_output/6_recognised.txt'.
Распознанный текст сохранен в 'output/easy_ocr_output/1_recognised.txt'.
Распознанный текст сохранен в 'output/easy_ocr_output/6_rotated_recognised.txt'.
Распознанный текст сохранен в 'output/easy_ocr_output/8_rotated_recognised.txt'.
Распознанный текст сохранен в 'output/easy_ocr_output/3_rotated_recognised.txt'.
Распознанный текст сохранен в 'output/easy_ocr_output/9_rotated_recognised.txt'.
Распознанный текст сохранен в 'output/easy_ocr_output/7_rotated_recognised.txt'.
Рас